In [27]:
from biobert_embedding.embedding import BiobertEmbedding
from utils import graph_tools
from multiprocessing import Pool
import torch

In [2]:
unique_drug_names = list(graph_tools.get_unique_drugs())

In [3]:
num_processes = 10
pool = Pool(num_processes)

## Original pip install biobert-embedding does not work (it's too old)

I forked the repo. Install using `pip install git+https://github.com/ariellubonja/biobert_embedding`

In [4]:
## Example 1
text = "Breast cancers with HER2 amplification have a higher risk of CNS metastasis and poorer prognosis."\

# Class Initialization (You can set default 'model_path=None' as your finetuned BERT model path while Initialization)
biobert = BiobertEmbedding()

Using existing models/pytorch_model.bin
Using existing models/config.json
Using existing models/vocab.txt


#### Basic example to make sure it works

In [5]:
word_embeddings = biobert.word_vector(text)
sentence_embedding = biobert.sentence_vector(text)

print("Text Tokens: ", biobert.tokens)
# Text Tokens:  ['breast', 'cancers', 'with', 'her2', 'amplification', 'have', 'a', 'higher', 'risk', 'of', 'cns', 'metastasis', 'and', 'poorer', 'prognosis', '.']

print ('Shape of Word Embeddings: %d x %d' % (len(word_embeddings), len(word_embeddings[0])))
# Shape of Word Embeddings: 16 x 768

print("Shape of Sentence Embedding = ",len(sentence_embedding))
# Shape of Sentence Embedding =  768

## Example 2
sentence_vector1 = biobert.sentence_vector('Breast cancers with HER2 amplification have a higher risk of CNS metastasis and poorer prognosis.')
sentence_vector2 = biobert.sentence_vector('Breast cancers with HER2 amplification are more aggressive, have a higher risk of CNS metastasis, and poorer prognosis.')

Text Tokens:  ['breast', 'cancers', 'with', 'her2', 'amplification', 'have', 'a', 'higher', 'risk', 'of', 'cns', 'metastasis', 'and', 'poorer', 'prognosis', '.']
Shape of Word Embeddings: 16 x 768
Shape of Sentence Embedding =  768


In [6]:
from scipy.spatial.distance import cosine as cosine_distance

cosine_sim = 1 - cosine_distance(sentence_vector1, sentence_vector2)
print('cosine similarity:', cosine_sim)
#cosine similarity: 0.992756187915802

cosine similarity: 0.9927560091018677


In [7]:
help(biobert.word_vector)

Help on method word_vector in module biobert_embedding.embedding:

word_vector(text, handle_oov=True, filter_extra_tokens=True) method of biobert_embedding.embedding.BiobertEmbedding instance



### Word Embeddings of Drugs.com drug names (without supporting text)

Find the drug names I've crawled [here](https://drive.google.com/drive/folders/1EO659a-tyjfXjKzHk-M0WreZBR14MgRM?usp=sharing)

<font color="orange">The cell below will try to handle Out of Vocabulary terms. This means that the embedding matrix is not a clear matrix, but will be a list of Tensors</font>

In [8]:
# This will take a while

# biobert_embeddings_oov = list(map(biobert.word_vector, unique_drug_names)) # Single-threaded

# Out of vocabulary will be split into smaller. May end up with multiple embeddings for each word
# parallelized
biobert_embeddings_oov = pool.map(biobert.word_vector, unique_drug_names)

pool.close()
pool.join()

In [10]:
len(biobert_embeddings_oov[0])

3

In [19]:
# These are the indices of the drug names that have only one embedding, i.e. the drug name is in the Vocabulary
vector_embd_indices = [i for i, num in enumerate(biobert_embeddings_oov) if len(num) == 1]
len(vector_embd_indices)

3310

In [17]:
len(biobert_embeddings_oov) == len(unique_drug_names)

True

In [45]:
vector_embeddings = torch.stack([biobert_embeddings_oov[i][0] for i in vector_embd_indices]).numpy()
corresponding_drug_names = [unique_drug_names[i] for i in vector_embd_indices]

In [44]:
vector_embeddings.numpy().shape

(3310, 768)

In [48]:
import pandas as pd

df = pd.DataFrame(vector_embeddings, index=corresponding_drug_names)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
trilafon,-2.494238,-5.982688,3.391733,-3.165889,-3.972550,-3.434661,2.095402,1.464997,6.532821,-1.086071,...,-9.374193,3.749851,1.355658,-2.396497,3.676836,1.897683,0.468625,6.310996,-9.883236,-1.279147
calcipotriene,4.240830,1.769997,-0.795535,-6.929158,5.979644,-0.761292,-1.656379,0.054974,0.432746,12.672398,...,-10.860700,-1.191983,-9.798771,-4.564957,-9.084117,-5.654516,-0.909690,10.118820,-6.920544,3.411890
fosinopril,2.749605,-11.437942,2.700483,-0.598100,9.610103,-13.214931,4.708538,7.986394,-0.904664,-3.148977,...,-8.771662,3.917134,-10.183059,5.208779,6.980632,0.385215,2.746163,-2.396154,-15.207134,-8.165861
nadostine,-2.403095,-5.212749,-2.980472,-4.289715,1.065220,-2.463279,2.731372,3.838523,-2.878551,3.916560,...,-5.626741,4.469468,0.329026,1.498180,-2.434177,-1.255941,0.073906,2.629568,-7.653620,3.064509
duavee,-2.835025,-6.494714,2.831769,-5.364807,4.386308,-0.174934,3.309786,-0.848888,2.427666,7.614682,...,0.924215,1.525765,-0.303652,3.410230,0.926888,-4.464722,2.695533,6.594471,-3.707011,-5.687425


## Evaluations

Top-K similarities

In [1]:
import faiss
import numpy as np


# Instantiate an index with the desired index type and dimensionality
index = faiss.IndexFlatIP(256)  # Cosine similarity, assuming 256-dimensional embeddings

biobert_embeddings_np = vector_embeddings

# Add biobert_embeddings_np to the index
index.add(biobert_embeddings_np)

# Specify the number of nearest neighbors to retrieve (k)
k = 5

# Perform the similarity search for each embedding
D, I = index.search(biobert_embeddings_np, k+1)  # Retrieve k+1 neighbors to exclude self

# D contains the similarities to the nearest neighbors (cosine similarity)
# I contains the indices of the nearest neighbors (excluding self)

# Iterate over each embedding
for i in range(biobert_embeddings_np.shape[0]):
    embedding = biobert_embeddings_np[i]
    nearest_indices = I[i][1:]  # Exclude self, start from index 1
    nearest_similarities = D[i][1:]  # Exclude self, start from index 1

    # Print the top-k nearest neighbors for the current embedding
    print(f"Embedding {i+1}:")
    for j, index in enumerate(nearest_indices):
        similarity = nearest_similarities[j]
        print(f"Nearest Neighbor {j+1}: Index {index}, Similarity {similarity}")
    print()


ModuleNotFoundError: No module named 'faiss'

## 3194 of the Drugs have vector embeddings, whereas the rest return a Matrix that we have to deal with using Pooling

Let's see the 3194

<font color="red">Most of the drugs give a matrix as their embedding, and not a vector.</font>

Need to try various pooling methods to get a vector from the matrix + see what is best

## TODO try ClinicalBERT https://github.com/EmilyAlsentzer/clinicalBERT

In [4]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

/Users/ariellubonja/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you 

In [5]:
encoded_inputs = tokenizer(unique_drug_names, padding=True, truncation=True, return_tensors='pt')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Macbook M1 Pro Runtimes

CPU - 1min 17 sec

MPS (torch 1.13) - Uses way too much memory (50GB+ - crashes) whereas CPU is max 13GB

MPS (torch 2.01) - 1min 15s - 18GB memory usage


<font color="red">Conclusion - not useful yet!</font>

In [7]:
%%timeit -r 3 -n 1

with torch.no_grad():
    model_output = model(**encoded_inputs)
    embeddings = model_output.last_hidden_state

1min 15s ± 8.66 s per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [ ]:
embeddings.shape

torch.Size([8, 768])